### AI2000 Hackathon

#### Authors: Tanay Yadav- AI20BTECH11026 , Tanmay Goyal- AI20BTECH11021

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# reading the data

train_df = pd.read_csv("/content/drive/MyDrive/CSV files/Hackathon- Machine Learning 2021 /train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/CSV files/Hackathon- Machine Learning 2021 /test.csv")
print(train_df.shape)
print(test_df.shape)

(51490, 42)
(77235, 42)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# train_df = pd.read_csv('/content/train.csv')
# test_df = pd.read_csv('/content/test.csv')

In [ ]:
train_df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location,Fault
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,NaN,NaN,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,466D8B-AC8C7,SUSPECTED MINOR INJURY,"N/A, WET",MD,7C2FEC-727F4,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,NaN,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,4028E4-C5384,NO APPARENT INJURY,"BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL,...",MD,67644F-64EF8,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,NaN,NaN,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NaN,NaN,889DBD-D57FF,NO APPARENT INJURY,NaN,DC,30F7A9-92C98,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583 7.3783,0
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,NaN,NaN,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,77155D-96AA7,POSSIBLE INJURY,NaN,MD,0F0C58-8E98F,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,"3.84,-70633",1
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,NaN,NaN,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,MEDICATION PRESENT,5E180B-42B0C,SUSPECTED MINOR INJURY,NaN,MD,4397C0-AEE83,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917 7.0183,0


In [ ]:
test_df.head()

,Id,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,0,MP810M,NaN,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,NaN,NaN,NaN,OTHER,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,ALCOHOL CONTRIBUTED,NaN,F05A35-C3C98,NO APPARENT INJURY,NaN,MD,DA635F-C3055,FUNCTIONAL,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MAKING RIGHT TURN,North,North,25,No,No,2004,LEXUS,RX330,NO MISUSE,38.988440,-77.127668,"3.84,-71763"
1,1,MP2001,NaN,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,NaN,GAITHERSBURG,NaN,SAME DIR REAR END,RAINING,WET,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,79C1D8-6A412,NO APPARENT INJURY,"RAIN, SNOW, WET",MD,DBC8E9-FC340,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,West,25,No,No,2001,MITSUBISHI,MIRAGE,NO MISUSE,39.094075,-77.205783,3.905 7.0733
2,2,MP6404,10125,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,NaN,NaN,NaN,SAME DIRECTION SIDESWIPE,NaN,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,30EC44-418A7,NO APPARENT INJURY,NaN,MD,32DA16-845E9,NO DAMAGE,TWO OCLOCK,TWO OCLOCK,AMBULANCE/EMERGENCY,MOVING CONSTANT SPEED,North,North,35,No,No,2016,FRIEGHTLINER,AMBULANCE,NO MISUSE,39.047180,-77.052688,"3.41,-70283"
3,3,MP0405,10270,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,"19600 FREDERICK RD, PAWN SHOP PARKING LOT",NaN,NaN,SAME DIRECTION SIDESWIPE,CLEAR,NaN,DAYLIGHT,NO CONTROLS,NaN,NaN,C7FA23-4267E,NO APPARENT INJURY,NaN,MD,4DC569-AB8EA,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,West,West,25,No,No,2004,FORD,CN,NaN,39.178007,-77.238475,3.7067 7.345
4,4,MP090D,10374,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,NaN,NaN,NaN,OTHER,CLEAR,DRY,DAYLIGHT,NO CONTROLS,UNKNOWN,NaN,B6023B-A34FF,NO APPARENT INJURY,NaN,NaN,B3BD7B-487E4,SUPERFICIAL,NINE OCLOCK,NINE OCLOCK,"MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4...",CHANGING LANES,North,North,55,No,No,2008,UTIL,TL,UNKNOWN,39.056134,-77.154043,3.5143 7.5022


In [ ]:
# column location can be split with delimiter as space or comma

new = train_df['Location'].str.split(",| " , expand = True)
train_df['Location_0'] = new[0].astype(float)
train_df['Location_1'] = new[1].astype(float)
train_df.drop(['Location'] , axis = 'columns', inplace = True)
train_df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Fault,Location_0,Location_1
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,NaN,NaN,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,466D8B-AC8C7,SUSPECTED MINOR INJURY,"N/A, WET",MD,7C2FEC-727F4,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,1,3.6383,7.4233
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,NaN,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,4028E4-C5384,NO APPARENT INJURY,"BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL,...",MD,67644F-64EF8,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,1,3.0862,7.3240
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,NaN,NaN,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NaN,NaN,889DBD-D57FF,NO APPARENT INJURY,NaN,DC,30F7A9-92C98,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,0,3.9583,7.3783
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,NaN,NaN,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,77155D-96AA7,POSSIBLE INJURY,NaN,MD,0F0C58-8E98F,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,1,3.8400,-70633.0000
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,NaN,NaN,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,MEDICATION PRESENT,5E180B-42B0C,SUSPECTED MINOR INJURY,NaN,MD,4397C0-AEE83,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,0,3.9917,7.0183


In [ ]:
# Repeating same steps on test data

new = test_df['Location'].str.split(",| " , expand = True)
test_df['Location_0'] = new[0].astype(float)
test_df['Location_1'] = new[1].astype(float)
test_df.drop(['Location'] , axis = 'columns', inplace = True)
test_df.head()

,Id,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location_0,Location_1
0,0,MP810M,NaN,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,NaN,NaN,NaN,OTHER,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,ALCOHOL CONTRIBUTED,NaN,F05A35-C3C98,NO APPARENT INJURY,NaN,MD,DA635F-C3055,FUNCTIONAL,ELEVEN OCLOCK,ELEVEN OCLOCK,PASSENGER CAR,MAKING RIGHT TURN,North,North,25,No,No,2004,LEXUS,RX330,NO MISUSE,38.988440,-77.127668,3.8400,-71763.0000
1,1,MP2001,NaN,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,NaN,GAITHERSBURG,NaN,SAME DIR REAR END,RAINING,WET,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,79C1D8-6A412,NO APPARENT INJURY,"RAIN, SNOW, WET",MD,DBC8E9-FC340,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,STOPPED IN TRAFFIC LANE,South,West,25,No,No,2001,MITSUBISHI,MIRAGE,NO MISUSE,39.094075,-77.205783,3.9050,7.0733
2,2,MP6404,10125,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,NaN,NaN,NaN,SAME DIRECTION SIDESWIPE,NaN,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,NaN,30EC44-418A7,NO APPARENT INJURY,NaN,MD,32DA16-845E9,NO DAMAGE,TWO OCLOCK,TWO OCLOCK,AMBULANCE/EMERGENCY,MOVING CONSTANT SPEED,North,North,35,No,No,2016,FRIEGHTLINER,AMBULANCE,NO MISUSE,39.047180,-77.052688,3.4100,-70283.0000
3,3,MP0405,10270,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,"19600 FREDERICK RD, PAWN SHOP PARKING LOT",NaN,NaN,SAME DIRECTION SIDESWIPE,CLEAR,NaN,DAYLIGHT,NO CONTROLS,NaN,NaN,C7FA23-4267E,NO APPARENT INJURY,NaN,MD,4DC569-AB8EA,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,West,West,25,No,No,2004,FORD,CN,NaN,39.178007,-77.238475,3.7067,7.3450
4,4,MP090D,10374,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,NaN,NaN,NaN,OTHER,CLEAR,DRY,DAYLIGHT,NO CONTROLS,UNKNOWN,NaN,B6023B-A34FF,NO APPARENT INJURY,NaN,NaN,B3BD7B-487E4,SUPERFICIAL,NINE OCLOCK,NINE OCLOCK,"MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4...",CHANGING LANES,North,North,55,No,No,2008,UTIL,TL,UNKNOWN,39.056134,-77.154043,3.5143,7.5022


In [ ]:
# We assert that both Location_0 and Location_1 coordinates are between -10 and 10

for index,row in train_df.iterrows():
    x = row['Location_0']
    y = row['Location_1']
    while(x < -10 or x > 10):
        x /= 10
    train_df.at[index , 'Location_0']  =  x

    while(y < -10 or y > 10):
         y /= 10
    train_df.at[index , 'Location_1']  =  y

for index,row in test_df.iterrows():
    x = row['Location_0']
    y = row['Location_1']
    while(x < -10 or x > 10):
        x /= 10
    test_df.at[index , 'Location_0']  =  x

    while(y < -10 or y > 10):
         y /= 10
    test_df.at[index , 'Location_1']  =  y



In [ ]:
train_df.head()

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,Municipality,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,Person ID,Injury Severity,Circumstance,Drivers License State,Vehicle ID,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Fault,Location_0,Location_1
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,NaN,NaN,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,466D8B-AC8C7,SUSPECTED MINOR INJURY,"N/A, WET",MD,7C2FEC-727F4,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,1,3.6383,7.4233
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,NaN,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,4028E4-C5384,NO APPARENT INJURY,"BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL,...",MD,67644F-64EF8,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,1,3.0862,7.3240
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,NaN,NaN,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NaN,NaN,889DBD-D57FF,NO APPARENT INJURY,NaN,DC,30F7A9-92C98,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,0,3.9583,7.3783
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,NaN,NaN,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,77155D-96AA7,POSSIBLE INJURY,NaN,MD,0F0C58-8E98F,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,1,3.8400,-7.0633
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,NaN,NaN,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,MEDICATION PRESENT,5E180B-42B0C,SUSPECTED MINOR INJURY,NaN,MD,4397C0-AEE83,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,0,3.9917,7.0183


In [ ]:
# We drop columns Report Number,Local Case Number, Person Id from both datasets
# We store column ID in test data for further use and then drop it

indices = test_df['Id']

train_df.drop(['Report Number', 'Local Case Number' , 'Vehicle ID'] , axis = 'columns' , inplace  = True)
test_df.drop(['Report Number',	'Local Case Number' , 'Id', 'Vehicle ID'] , axis = 'columns' , inplace  = True)

In [ ]:
# Since most columns are catagorical, the idea is to drop columns with NaNs in them
# But since many of the columns are having NaNs, replace them with most common value
# Not before dropping columns with more than 20000 Nans
pd.isna(train_df).sum() 

for column in train_df.columns:
    if (pd.isna(train_df).sum() > 20000 )[column] ==True:
        train_df.drop([column] , axis = 'columns' , inplace = True)

train_df = train_df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [ ]:
# retrieving Fault column and storing it as target variable
y_train = np.array(train_df['Fault'])

train_df.drop(['Fault'] , axis = 'columns' , inplace = True)

In [ ]:
# rearranging the columns of test_df in the same order as train_df
test_df = test_df[train_df.columns]

# filling in the NaNs of a column of test_df with most repeated values from 
# same column of train_df
for column in test_df.columns:
    test_df[column].fillna(train_df[column].mode()[0] , inplace = True)


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
train_df.head()

,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Person ID,Injury Severity,Drivers License State,Vehicle Damage Extent,Vehicle First Impact Location,Vehicle Second Impact Location,Vehicle Body Type,Vehicle Movement,Vehicle Continuing Dir,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location_0,Location_1
0,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,466D8B-AC8C7,SUSPECTED MINOR INJURY,MD,DESTROYED,TWELVE OCLOCK,TWELVE OCLOCK,PICKUP TRUCK,MOVING CONSTANT SPEED,South,Unknown,40,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383,7.4233
1,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,4028E4-C5384,NO APPARENT INJURY,MD,DISABLING,TWELVE OCLOCK,TWELVE OCLOCK,PASSENGER CAR,ACCELERATING,South,South,25,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862,7.3240
2,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NONE DETECTED,889DBD-D57FF,NO APPARENT INJURY,DC,SUPERFICIAL,SIX OCLOCK,SIX OCLOCK,TRANSIT BUS,PARKING,West,West,35,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583,7.3783
3,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,77155D-96AA7,POSSIBLE INJURY,MD,FUNCTIONAL,TWELVE OCLOCK,TWO OCLOCK,PASSENGER CAR,MOVING CONSTANT SPEED,South,South,25,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,3.8400,-7.0633
4,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,5E180B-42B0C,SUSPECTED MINOR INJURY,MD,DISABLING,SIX OCLOCK,SIX OCLOCK,PASSENGER CAR,SLOWING OR STOPPING,South,South,40,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917,7.0183


In [ ]:
# we change the crash date/time column into a column which tells us if
# the crash was during hours AM or PM

# extracting AM and PM from Crash Date/Time column

new = train_df['Crash Date/Time'].str.split(" " ,  expand = True)
train_df['time'] = new[2]

# drop Crash Date/Time column
train_df.drop(['Crash Date/Time'] , axis = 'columns' , inplace = True)

# considering more prioirity to PM
train_df['time'] = train_df['time'].replace(['AM' , 'PM'] , [0,1])




In [ ]:
# repeating the same steps on Crash Date/Time on test_df

# we change the crash date/time column into a column which tells us if
# the crash was during hours AM or PM

# extracting AM and PM from Crash Date/Time column

new = test_df['Crash Date/Time'].str.split(" " ,  expand = True)
test_df['time'] = new[2]

# drop Crash Date/Time column
test_df.drop(['Crash Date/Time'] , axis = 'columns' , inplace = True)

# considering more prioirity to PM
test_df['time'] = test_df['time'].replace(['AM' , 'PM'] , [0,1])




In [ ]:
# finding the number of unique values per column with dtype==object

for column in train_df.columns:
    if train_df[column].dtypes == object:
        print(column , len(pd.unique(train_df[column])))

Agency Name 10
ACRS Report Type 3
Route Type 10
Road Name 2226
Cross-Street Type 10
Cross-Street Name 4525
Collision Type 18
Weather 12
Surface Condition 11
Light 8
Traffic Control 11
Driver Substance Abuse 11
Person ID 49209
Injury Severity 5
Drivers License State 65
Vehicle Damage Extent 7
Vehicle First Impact Location 16
Vehicle Second Impact Location 16
Vehicle Body Type 30
Vehicle Movement 22
Vehicle Continuing Dir 5
Vehicle Going Dir 5
Driverless Vehicle 2
Parked Vehicle 2
Vehicle Make 886
Vehicle Model 3128
Equipment Problems 8


In [ ]:
# we drop columns with more than 20 unique values

for column in train_df.columns:
    if len(pd.unique(train_df[column])) > 50:
        train_df.drop(column, axis = 'columns' , inplace = True)

In [ ]:
# keeping same columns in test_df
test_df = test_df[train_df.columns]

# comparing unique values per column in test and train data:
for column in test_df.columns:
    if test_df[column].dtypes == object:
            print(column , ", train= " ,len(pd.unique(train_df[column])) , ", test = ", len(pd.unique(test_df[column])))

Agency Name , train=  10 , test =  10
ACRS Report Type , train=  3 , test =  3
Route Type , train=  10 , test =  10
Cross-Street Type , train=  10 , test =  10
Collision Type , train=  18 , test =  18
Weather , train=  12 , test =  12
Surface Condition , train=  11 , test =  11
Light , train=  8 , test =  8
Traffic Control , train=  11 , test =  11
Driver Substance Abuse , train=  11 , test =  11
Injury Severity , train=  5 , test =  5
Vehicle Damage Extent , train=  7 , test =  7
Vehicle First Impact Location , train=  16 , test =  16
Vehicle Second Impact Location , train=  16 , test =  16
Vehicle Body Type , train=  30 , test =  30
Vehicle Movement , train=  22 , test =  22
Vehicle Continuing Dir , train=  5 , test =  5
Vehicle Going Dir , train=  5 , test =  5
Driverless Vehicle , train=  2 , test =  2
Parked Vehicle , train=  2 , test =  2
Equipment Problems , train=  8 , test =  10


In [ ]:
# dropping Equipment problems because testing data has more unique values than
# training data

train_df.drop(['Equipment Problems'] , axis = 'columns' , inplace = True)
test_df.drop(['Equipment Problems'] , axis = 'columns' , inplace = True)

In [ ]:
# one hot encoding

train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)

print(train_df.shape, test_df.shape)

(51490, 216) (77235, 216)


In [ ]:
X_train = np.array(train_df)
X_test = np.array(test_df)

### Running the models

In [ ]:
model = KNeighborsClassifier(n_neighbors=7)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)


predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('KNN.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)


In [ ]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('RFC_1000.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)

In [ ]:
model = GradientBoostingClassifier(n_estimators=2750)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('GBC_2750.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)

In [ ]:
model =  MLPClassifier( max_iter=700)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('MLP_700.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)

In [30]:
model = XGBClassifier(max_depth = 200, subsample = 0.75)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('XGB_2.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)

In [ ]:
model = LGBMClassifier(n_estimators = 1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

predictions = []
for i in range(len(indices)):
    predictions.append([indices[i] , y_pred[i]])

predictions = np.array(predictions)

pred_df = pd.DataFrame(predictions , columns = ['Id' , 'Fault'])
pred_df

pred_df.to_csv('LGBM_1000.csv' , columns = ['Id' , 'Fault'] , header = ['Id' , 'Fault'] , index = False)

### Conclusion

The final models we used was a Gradient Boosting Classifier with the number of estimators as 2500, and a Light Gradient Boosting Model with n_estimators = 1000.


The reason why we chose Gradient Boosting Classifier is because it combines various weak learners, by improving on the mistakes made by the previous models, and combines the results of all models. It is robust to over-fitting. It is a greedy algorithm, choosing trees based on best split, and the weak learners can be constrained, to ensure they remain week, but greedy.

Light Gradient Boosting Model is an algorithm,that significantly cuts on memory, and is extremely fast-paced, with note-worthy accuracies. It splits trees leaf-wise, rather than level-wise    , and hence, obtains better accuracies compared to many of the other models. 

The reason why we chose these models is because of the high number of features in our data due to the one hot encoding done, and an iterative approach to finding the pattern seemed like the best idea.
